In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as pplt
from matplotlib.font_manager import FontProperties
from sklearn.manifold import TSNE
from scipy import stats
import tensorflow as tf

In [2]:
# Loading the data
df = pd.read_csv("/Users/isabel/Desktop/koito/student-mat.csv", sep=';')
#df.info()
bin_cols = [ 'school', 'sex', 'address', 'famsize', 'Pstatus','schoolsup','famsup','activities','paid','internet','nursery','higher','romantic']
enc = preprocessing.LabelEncoder()

#to convert into numbers
for colName in bin_cols:
    df[colName] = enc.fit_transform(df[colName])
    #print max(df[colName])
#df.tail()


multi_cols = ['Mjob','Fjob','guardian','reason']

for Ncols in multi_cols:
    if Ncols in df.columns:
        hoe = pd.get_dummies(df[Ncols])

        for idxs in range(len(hoe.columns)):
            str1 = hoe.columns[idxs]
            str2 = Ncols+'_'+hoe.columns[idxs]
            hoe=hoe.rename(columns ={str1:str2})
        #print hoe.columns
        df = df.drop(Ncols, axis=1)
        df = df.join(hoe.astype(int))
df.head()


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,guardian_father,guardian_mother,guardian_other,reason_course,reason_home,reason_other,reason_reputation
0,0,0,18,1,0,0,4,4,2,2,...,0,0,1,0,1,0,1,0,0,0
1,0,0,17,1,0,1,1,1,1,2,...,1,0,0,1,0,0,1,0,0,0
2,0,0,15,1,1,1,1,1,1,2,...,1,0,0,0,1,0,0,0,1,0
3,0,0,15,1,0,1,4,2,1,3,...,0,1,0,0,1,0,0,1,0,0
4,0,0,16,1,0,1,3,3,1,2,...,1,0,0,1,0,0,0,1,0,0


In [ ]:
# Plot all relative numbers vs year
fontP = FontProperties()
fontP.set_size('small')
fig = pplt.figure()
ax = pplt.subplot(111)

#for item in newlistRate:
ax.plot(df['age'],df['Medu'], "or")
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

ax.legend(title="Alc data",loc='center left', prop = fontP, bbox_to_anchor=(1, 0.5))
pplt.show()

In [ ]:
# Plot original TSNE

model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
transX = model.fit_transform(df) 
fig = pplt.figure()
pplt.scatter(transX[:, 0], transX[:, 1], c=df['G1'], cmap=pplt.cm.Spectral)
pplt.title("t-SNE of classes")
pplt.show()

# absences are circular with middle highest absence rate students, right bottom group low absence rate

In [3]:
#More data cleaning and processing

if 'Dalc' in df.columns:
    Y_dataD = df['Dalc']
    Y_dataD = Y_dataD[0:len(Y_dataD)].values
    df = df.drop('Dalc', axis=1)
if 'Walc' in df.columns:
    Y_dataW = df['Walc']
    Y_dataW = Y_dataW[0:len(Y_dataW)].values
    df = df.drop('Walc', axis=1)
#Y = df.join(hoe.astype(int))
dfs = preprocessing.scale(df)
#print dfs[0,1]
print df.columns
#print len(dfs[0])
num_feat = len(dfs[0])
num_samples = len(dfs)
    
Y_data = (2*Y_dataW+5*Y_dataD)/7
#print Y_data[0:100]

Index([u'school', u'sex', u'age', u'address', u'famsize', u'Pstatus', u'Medu',
       u'Fedu', u'traveltime', u'studytime', u'failures', u'schoolsup',
       u'famsup', u'paid', u'activities', u'nursery', u'higher', u'internet',
       u'romantic', u'famrel', u'freetime', u'goout', u'health', u'absences',
       u'G1', u'G2', u'G3', u'Mjob_at_home', u'Mjob_health', u'Mjob_other',
       u'Mjob_services', u'Mjob_teacher', u'Fjob_at_home', u'Fjob_health',
       u'Fjob_other', u'Fjob_services', u'Fjob_teacher', u'guardian_father',
       u'guardian_mother', u'guardian_other', u'reason_course', u'reason_home',
       u'reason_other', u'reason_reputation'],
      dtype='object')


In [ ]:
#Processed Data TSNE
model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
transX = model.fit_transform(dfs) 
fig = pplt.figure()
pplt.scatter(transX[:, 0], transX[:, 1], c=Y_dataW, cmap=pplt.cm.Spectral)
pplt.title("t-SNE of classes")
pplt.show()

In [ ]:
# Statsistical tests
# Dalc and Walc are strongly correlated, although not normaldist, p-value not so accurate since data is small
print stats.shapiro(Y_data)
print stats.pearsonr(Y_data, Y_dataW)

In [ ]:
# Histogram plot 
pplt.hist(df['age'])
pplt.title("Histogram")
pplt.xlabel("Value")
pplt.ylabel("Frequency")

fig = pplt.gcf()
pplt.show()
#Shows no normal distribution

In [15]:
# Data and model preparation for NN learning

# pediction of Dalc
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

# Our model is a standard 1-hidden-layer multi-layer-perceptron with ReLU
# activation. The softmax (which turns arbitrary real-valued outputs into
# probabilities) gets applied in the cost function.
def model(X, w_h, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)
# How many units in the hidden layer + batch size
NUM_HIDDEN = 200
BATCH_SIZE = 128
# Our variables. The input has width NUM_Feat, and the output has width 5
X = tf.placeholder("float", [None, num_feat])
Y = tf.placeholder("float", [None, 5])


#w = init_weights([num_feat, 1]) # like in linear regression, we need a shared variable weight matrix for logistic regression
w_h = init_weights([num_feat, NUM_HIDDEN])
w_o = init_weights([NUM_HIDDEN, 5])

#y = tf.matmul(tf.nn.relu(tf.matmul(X, w_h)), w_o)
y = tf.matmul(tf.nn.relu(tf.matmul(X, w_h)), w_o)
# Predict y given x using the model.
py_x = model(X, w_h, w_o)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)
predict_op = tf.argmax(py_x, 1) # at predict time, evaluate the argmax 

p = np.random.permutation(range(len(dfs)))
allX, allY = dfs[p], Y_data[p]
border = int(0.8*num_samples)
hotY = pd.get_dummies(allY)
hoty = hotY.as_matrix()

trX = allX[:border]
teX = allX[border:]
trY = hoty[:border]
teY = hoty[border:]


In [16]:
# Launch the graph in an interactive session
sess = tf.InteractiveSession()
tf.initialize_all_variables().run()

for epoch in range(400):
    # Shuffle the data before each training iteration.
    p = np.random.permutation(range(len(trX)))
    trX, trY = trX[p], trY[p]

    # Train in batches 
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE        
        sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})

        
    # And print the current accuracy on the training data.
    if epoch % 20==0:
        #compute accuracy
        correct_prediction = tf.equal(tf.cast(tf.argmax(y,1), tf.float32),tf.cast(tf.argmax(Y,1), tf.float32) )
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        result = sess.run(accuracy, feed_dict={X: trX[start:end], Y: trY[start:end]})
        print 'Run {}, {}'.format(epoch+1, result)
        
        # average weight filter
        wb = sess.run(w_h)
        print np.mean(wb) 



Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x1193f0650>> ignored


Run 1, 0.566666662693
-1.03002e-05
Run 21, 0.699999988079
-0.000193513
Run 41, 0.633333325386
-0.000996464
Run 61, 0.733333349228
-0.00150201
Run 81, 0.683333337307
-0.00186838
Run 101, 0.716666638851
-0.00224142
Run 121, 0.633333325386
-0.00250458
Run 141, 0.733333349228
-0.00272843
Run 161, 0.699999988079
-0.00291935
Run 181, 0.866666674614
-0.00312305
Run 201, 0.800000011921
-0.00328836
Run 221, 0.866666674614
-0.00344558
Run 241, 0.883333325386
-0.00358987
Run 261, 0.916666686535
-0.00371235
Run 281, 0.966666638851
-0.00383605
Run 301, 0.899999976158
-0.00392617
Run 321, 0.949999988079
-0.00399846
Run 341, 0.916666686535
-0.00405571
Run 361, 0.949999988079
-0.00411178
Run 381, 0.933333337307
-0.00415202


In [17]:
# Check test error
correct_prediction = tf.equal(tf.cast(tf.argmax(y,1), tf.float32),tf.cast(tf.argmax(Y,1), tf.float32) )
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
result = sess.run(accuracy, feed_dict={X: teX[:], Y: teY[:]})
print 'Evaluation on test data {}'.format(result)

Evaluation on test data 0.645569622517


In [7]:
# Close the Session when we're done.
sess.close()

In [ ]:
#https://www.researchgate.net/publication/296695247_USING_DATA_MINING_TO_PREDICT_SECONDARY_SCHOOL_STUDENT_ALCOHOL_CONSUMPTION
# paper also gives most relevant features by random forest
# Only 92% accuracy achieved